# Ratehawk data validation process.

In [10]:
import pandas as pd
import os

In [11]:
file = 'D:/ratehawk_data_upload_database/partner_feed_en_v3_full.csv'

### Count total of rows.

In [5]:
import pandas as pd

file = 'D:/ratehawk_data_upload_database/partner_feed_en_v3_full.csv'
total_rows = 0

# Read in chunks and count the rows in each chunk
for chunk in pd.read_csv(file, chunksize=100000): 
    total_rows += len(chunk)

# print(f"Total number of rows: {total_rows}")
total number of rows = 2620000

SyntaxError: invalid syntax (794325839.py, line 11)

### Take only headers row only.

In [3]:
import pandas as pd
import os

file = 'D:/ratehawk_data_upload_database/partner_feed_en_v3_full.csv'


df = pd.read_csv(file, nrows = 1)
headers = df.columns.tolist()
print(headers)

# df_header = pd.DataFrame(columns=headers)

# print(df_header)


['address', 'amenity_groups', 'check_in_time', 'check_out_time', 'description_struct', 'id', 'hid', 'images', 'images_ext', 'kind', 'latitude', 'longitude', 'name', 'phone', 'policy_struct', 'postal_code', 'room_groups', 'region', 'star_rating', 'email', 'serp_filters', 'deleted', 'is_closed', 'is_gender_specification_required', 'metapolicy_struct', 'metapolicy_extra_info', 'star_certificate', 'facts', 'payment_methods', 'hotel_chain', 'front_desk_time_start', 'front_desk_time_end', 'keys_pickup']


In [ ]:
df = pd.read_csv(file, nrows=5)
print(df)

### Show selectted column information

In [ ]:
import pandas as pd

file = 'D:/ratehawk_data_upload_database/partner_feed_en_v3_full.csv'

desired_columns = [
    'address', 'amenity_groups', 'check_in_time', 'check_out_time', 
    'description_struct', 'id', 'hid', 'images', 'images_ext', 
    'kind', 'latitude', 'longitude', 'name', 'phone', 'policy_struct'
]

df = pd.read_csv(file, usecols=desired_columns)

print(df.head(2))  


### Create column select csv file header row

In [45]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Float, Boolean, String
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import text

# Database connection setup (replace with your actual database URL)
DATABASE_URL = "mysql+pymysql://root:@localhost/csvdata01_02102024"
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()

# Load column headers from the CSV file
file = 'D:/ratehawk_data_upload_database/partner_feed_en_v3_full.csv'
df = pd.read_csv(file, nrows=1)
headers = df.columns.tolist()

# Use the existing 'ratehawk' table
metadata = MetaData()
ratehawk_table = Table('ratehawk', metadata, autoload_with=engine)

# Add new columns to the table dynamically
for column_name in headers:
    if column_name not in ratehawk_table.columns:
        # Define the column type
        if column_name in ['latitude', 'longitude', 'star_rating']:
            new_column = Column(column_name, Float)
        elif column_name in ['is_closed', 'is_gender_specification_required', 'deleted']:
            new_column = Column(column_name, Boolean)
        else:
            # For String columns, set a default length (e.g., 255)
            new_column = Column(column_name, String(255))

        # Prepare and execute the ALTER TABLE statement manually
        with engine.connect() as conn:
            alter_statement = f"ALTER TABLE ratehawk ADD COLUMN {column_name} {new_column.type.compile(dialect=engine.dialect)}"
            conn.execute(text(alter_statement))  # Use `text()` to convert the string to an executable object

# Commit the changes to the session and close
session.commit()
session.close()

print("Columns added successfully.")


Columns added successfully.


In [ ]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.sql import text
import numpy as np

# Database connection setup
DATABASE_URL = "mysql+pymysql://root:@localhost/csvdata01_02102024"
engine = create_engine(DATABASE_URL)

# Define chunk size (adjust based on available memory)
chunk_size = 10000  # Adjust as needed based on your system's memory

# Read the CSV in chunks and process each chunk
file = 'D:/ratehawk_data_upload_database/partner_feed_en_v3_full.csv'

# Use the existing 'ratehawk' table
metadata = MetaData()
ratehawk_table = Table('ratehawk', metadata, autoload_with=engine)

def handle_nan_values(data):
    """
    Replace NaN values in the dictionary with None.
    This ensures compatibility with MySQL.
    """
    for key, value in data.items():
        if isinstance(value, float) and (pd.isna(value) or np.isnan(value)):
            data[key] = None  # Replace NaN with None
    return data

# Process CSV in chunks
for chunk in pd.read_csv(file, chunksize=chunk_size):
    # Iterate over each row in the chunk
    with engine.connect() as conn:
        # Ensure autocommit is enabled for each row
        conn.execution_options(autocommit=True)

        for _, row in chunk.iterrows():
            # Convert the row to a dictionary
            data = row.to_dict()

            # Ensure that data matches the table columns
            filtered_data = {k: v for k, v in data.items() if k in ratehawk_table.columns}

            # Handle NaN values before inserting
            filtered_data = handle_nan_values(filtered_data)

            # Prepare the insert statement
            insert_statement = ratehawk_table.insert().values(filtered_data)

            # Execute and commit each insert immediately
            conn.execute(insert_statement)

print("CSV data uploaded successfully.")


In [55]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import text
import numpy as np

# Database connection setup
DATABASE_URL = "mysql+pymysql://root:@localhost/csvdata01_02102024"
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()

# Define chunk size (adjust based on available memory)
chunk_size = 10000  # Adjust as needed based on your system's memory

# Read the CSV in chunks and process each chunk
file = 'D:/ratehawk_data_upload_database/partner_feed_en_v3_full.csv'

# Use the existing 'ratehawk' table
metadata = MetaData()
ratehawk_table = Table('ratehawk', metadata, autoload_with=engine)

def handle_nan_values(data):
    """
    Replace NaN values in the dictionary with None.
    This ensures compatibility with MySQL.
    """
    for key, value in data.items():
        if isinstance(value, float) and (pd.isna(value) or np.isnan(value)):
            data[key] = None  # Replace NaN with None
    return data

# Process CSV in chunks
for chunk in pd.read_csv(file, chunksize=chunk_size):
    # Iterate over each row in the chunk
    with engine.connect() as conn:
        for _, row in chunk.iterrows():
            # Convert the row to a dictionary
            data = row.to_dict()

            # Ensure that data matches the table columns
            filtered_data = {k: v for k, v in data.items() if k in ratehawk_table.columns}

            # Handle NaN values before inserting
            filtered_data = handle_nan_values(filtered_data)

            # Prepare the insert statement
            insert_statement = ratehawk_table.insert().values(filtered_data)

            # Execute the insert statement for each row
            conn.execute(insert_statement)

# Commit the changes to the session and close
session.commit()
session.close()

print("CSV data uploaded successfully.")


CSV data uploaded successfully.


### Here use Begin()

In [4]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table
import numpy as np

# Database connection setup
DATABASE_URL = "mysql+pymysql://root:@localhost/csvdata01_02102024"
engine = create_engine(DATABASE_URL)

# Define chunk size (adjust based on available memory)
chunk_size = 10000  # Adjust as needed based on your system's memory

# Read the CSV in chunks and process each chunk
file = 'D:/ratehawk_data_upload_database/partner_feed_en_v3_full.csv'

# Use the existing 'ratehawk' table
metadata = MetaData()
ratehawk_table = Table('ratehawk', metadata, autoload_with=engine)

def handle_nan_values(data):
    """
    Replace NaN values in the dictionary with None.
    This ensures compatibility with MySQL.
    """
    for key, value in data.items():
        if isinstance(value, float) and (pd.isna(value) or np.isnan(value)):
            data[key] = None  # Replace NaN with None
    return data

# Process CSV in chunks
for chunk in pd.read_csv(file, chunksize=chunk_size):
    # Iterate over each row in the chunk
    with engine.connect() as conn:
        for _, row in chunk.iterrows():
            # Convert the row to a dictionary
            data = row.to_dict()

            # Ensure that data matches the table columns
            filtered_data = {k: v for k, v in data.items() if k in ratehawk_table.columns}

            # Handle NaN values before inserting
            filtered_data = handle_nan_values(filtered_data)

            # Prepare and execute the insert statement in an individual transaction
            with conn.begin() as transaction:
                insert_statement = ratehawk_table.insert().values(filtered_data)
                conn.execute(insert_statement)

print("CSV data uploaded successfully.")


CSV data uploaded successfully.


### Convert zstd file to csv file.

In [4]:
import zstandard as zstd

def extract_zst_to_csv(zst_file, csv_file):
    try:
        print("Starting decompression...")
        with open(zst_file, 'rb') as compressed:
            print(f"Opened {zst_file} for decompression.")
            dctx = zstd.ZstdDecompressor()

            # Stream decompression
            with open(csv_file, 'wb') as out_csv:
                print(f"Opening {csv_file} for writing decompressed data.")
                dctx.copy_stream(compressed, out_csv)
                print("Decompression completed successfully.")
                
        return csv_file

    except zstd.ZstdError as e:
        print(f"Zstandard decompression error: {e}")
        return None
    except Exception as e:
        print(f"Error during decompression: {e}")
        return None

zst_file_path = "D:/ratehawk_data_upload_database/partner_feed_en_v3.jsonl.zst"
csv_file = "D:/ratehawk_data_upload_database/partner_feed_en_v3.jsonl.csv"

data = extract_zst_to_csv(zst_file_path, csv_file)

if data:
    print(f"Decompression complete, CSV saved at: {data}")
else:
    print("Decompression failed.")


Starting decompression...
Opened D:/ratehawk_data_upload_database/partner_feed_en_v3.jsonl.zst for decompression.
Opening D:/ratehawk_data_upload_database/partner_feed_en_v3.jsonl.csv for writing decompressed data.
Decompression completed successfully.
Decompression complete, CSV saved at: D:/ratehawk_data_upload_database/partner_feed_en_v3.jsonl.csv


### Execution data selected rows.

In [ ]:
import pandas as pd
import json

def convert_jsonl_to_csv(jsonl_file, csv_file, num_rows=1000):
    try:
        rows = []
        
        # Open the JSONL file and read it line-by-line
        with open(jsonl_file, 'r', encoding='ISO-8859-1') as file:
            for i, line in enumerate(file):
                if i >= num_rows:
                    break  # Stop after reaching the required number of rows

                try:
                    # Parse each line as JSON
                    row_data = json.loads(line)
                    rows.append(row_data)
                except json.JSONDecodeError as e:
                    print(f"Skipping malformed line {i}: {e}")

        # Convert the list of dictionaries (rows) to a DataFrame
        df = pd.DataFrame(rows)
        
        # Save the DataFrame to a CSV file
        df.to_csv(csv_file, index=False, encoding='utf-8')
        print(f"Successfully converted to CSV and saved at: {csv_file}")
        return df

    except Exception as e:
        print(f"Error processing JSONL to CSV: {e}")
        return None

jsonl_file = 'D:/ratehawk_data_upload_database/partner_feed_en_v3.jsonl.csv'  
csv_output_file = 'D:/ratehawk_data_upload_database/partner_feed_en_v3_subset.csv'  

# Execute the conversion and check output
data = convert_jsonl_to_csv(jsonl_file, csv_output_file)
print(data if data is not None else "Conversion failed.")


### Convert all data to json file.

In [8]:
import pandas as pd
import json

def convert_jsonl_to_csv(jsonl_file, csv_file, num_rows=None):
    try:
        rows = []
        processed_count = 0
        skipped_count = 0
        
        # Open the JSONL file and read it line-by-line
        with open(jsonl_file, 'r', encoding='ISO-8859-1') as file:
            line_count = 0
            for i, line in enumerate(file):
                if num_rows and line_count >= num_rows:
                    break 
                line_count += 1

                try:
                    # Parse each line as JSON
                    row_data = json.loads(line)
                    rows.append(row_data)
                    processed_count += 1 
                except json.JSONDecodeError as e:
                    skipped_count += 1 
                    print(f"Skipping malformed line {i}: {e}")
                except Exception as e:
                    skipped_count += 1  
                    print(f"Unexpected error in line {i}: {e}")

        if not rows:
            print("No valid data found in the JSONL file.")
            return None

        # Convert the list of dictionaries (rows) to a DataFrame
        df = pd.DataFrame(rows)
        
        # Save the DataFrame to a CSV file
        df.to_csv(csv_file, index=False, encoding='utf-8')
        print(f"Successfully converted to CSV and saved at: {csv_file}")
        print(f"Processed {processed_count} rows, skipped {skipped_count} rows.")
        return df

    except Exception as e:
        print(f"Error processing JSONL to CSV: {e}")
        return None

jsonl_file = 'D:/ratehawk_data_upload_database/partner_feed_en_v3.jsonl.csv'  
csv_output_file = 'D:/ratehawk_data_upload_database/partner_feed_en_v3_subset_data_all.csv' 

# Execute the conversion and check output
data = convert_jsonl_to_csv(jsonl_file, csv_output_file)
print(data if data is not None else "Conversion failed.")


Error processing JSONL to CSV: 
Conversion failed.


In [11]:
import pandas as pd
import json
import csv

def convert_jsonl_to_csv(jsonl_file, csv_file, batch_size=100):
    try:
        processed_count = 0
        skipped_count = 0
        
        # Open the CSV file to write
        with open(csv_file, 'w', newline='', encoding='utf-8') as csv_output:
            writer = None  # CSV writer will be initialized after the first batch
            rows = []

            # Open the JSONL file and read it line-by-line
            with open(jsonl_file, 'r', encoding='ISO-8859-1') as file:
                for i, line in enumerate(file):
                    try:
                        # Parse each line as JSON
                        row_data = json.loads(line)
                        rows.append(row_data)
                        processed_count += 1

                        # If batch is ready, write to CSV
                        if len(rows) >= batch_size:
                            df = pd.DataFrame(rows)
                            if writer is None:
                                # Initialize writer and write header for the first batch
                                writer = csv.DictWriter(csv_output, fieldnames=df.columns)
                                writer.writeheader()
                            writer.writerows(df.to_dict(orient='records'))
                            rows = []  # Clear the batch

                    except json.JSONDecodeError as e:
                        skipped_count += 1
                        print(f"Skipping malformed line {i}: {e}")
                    except Exception as e:
                        skipped_count += 1
                        print(f"Unexpected error in line {i}: {e}")

            # Write any remaining rows in the last batch
            if rows:
                df = pd.DataFrame(rows)
                if writer is None:
                    writer = csv.DictWriter(csv_output, fieldnames=df.columns)
                    writer.writeheader()
                writer.writerows(df.to_dict(orient='records'))

        print(f"Successfully converted to CSV and saved at: {csv_file}")
        print(f"Processed {processed_count} rows, skipped {skipped_count} rows.")
        return processed_count, skipped_count

    except Exception as e:
        print(f"Error processing JSONL to CSV: {e}")
        return None

# File paths
jsonl_file = 'D:/ratehawk_data_upload_database/partner_feed_en_v3.jsonl.csv'  
csv_output_file = 'D:/ratehawk_data_upload_database/partner_feed_en_v3_full.csv'  

# Execute the conversion and check output
processed, skipped = convert_jsonl_to_csv(jsonl_file, csv_output_file)
if processed is not None:
    print(f"Conversion completed with {processed} rows processed and {skipped} rows skipped.")
else:
    print("Conversion failed.")


Successfully converted to CSV and saved at: D:/ratehawk_data_upload_database/partner_feed_en_v3_full.csv
Processed 2606315 rows, skipped 0 rows.
Conversion completed with 2606315 rows processed and 0 rows skipped.


## Get only header

In [5]:
import pandas as pd
import json

def convert_jsonl_to_csv_with_header(jsonl_file, csv_file, num_rows=2):
    try:
        rows = []
        
        # Open the JSONL file and read it line-by-line
        with open(jsonl_file, 'r', encoding='ISO-8859-1') as file:
            for i, line in enumerate(file):
                if i >= num_rows:
                    break 

                try:
                    # Parse each line as JSON
                    row_data = json.loads(line)
                    rows.append(row_data)
                except json.JSONDecodeError as e:
                    print(f"Skipping malformed line {i}: {e}")

        # Convert the list of dictionaries (rows) to a DataFrame
        df = pd.DataFrame(rows)
        
        # Extract headers (column names)
        headers = df.columns.tolist()

        # Create an empty DataFrame with only the headers
        df_header = pd.DataFrame(columns=headers)
        
        # Save the DataFrame with only headers to a CSV file
        df_header.to_csv(csv_file, index=False, encoding='utf-8')
        # print(f"Successfully created header-only CSV and saved at: {csv_file}")
        return df_header

    except Exception as e:
        print(f"Error processing JSONL to header-only CSV: {e}")
        return None

jsonl_file = 'D:/ratehawk_data_upload_database/partner_feed_en_v3.jsonl.csv' 
csv_output_file = 'D:/ratehawk_data_upload_database/partner_feed_en_v3_header.csv'  

# Execute the conversion and check output
header_data = convert_jsonl_to_csv_with_header(jsonl_file, csv_output_file)
print(header_data if header_data is not None else "Header extraction failed.")


Empty DataFrame
Columns: [address, amenity_groups, check_in_time, check_out_time, description_struct, id, hid, images, images_ext, kind, latitude, longitude, name, phone, policy_struct, postal_code, room_groups, region, star_rating, email, serp_filters, deleted, is_closed, is_gender_specification_required, metapolicy_struct, metapolicy_extra_info, star_certificate, facts, payment_methods, hotel_chain, front_desk_time_start, front_desk_time_end, keys_pickup]
Index: []

[0 rows x 33 columns]


### Get data from localdb

In [27]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table
import numpy as np
import json

# Database connection setup
DATABASE_URL = "mysql+pymysql://root:@localhost/csvdata01_02102024"
engine = create_engine(DATABASE_URL)

table = "ratehawk"

query = f"SELECT * FROM {table} LIMIT 1 OFFSET 4"

with engine.connect() as conn:
    first_row = pd.read_sql_query(query, conn)



### Convert table row into the json file

In [ ]:
first_row_dict = first_row.astype(str).to_dict(orient="records")[0] 
# first_row_dict = first_row.to_dict(orient="records")[0] 
first_row_json = json.dumps(first_row_dict, indent=4) 

print(first_row_json)

### Get a selectted row data

In [ ]:
query = "SELECT amenity_groups FROM ratehawk WHERE my_id = '3199216';"

with engine.connect() as conn:
    column = pd.read_sql_query(query, conn)
    data = column.iloc[0,0]
    print(data)


### Create json file.

In [48]:
import json
import pandas as pd
from sqlalchemy import create_engine

# Database connection setup
DATABASE_URL = "mysql+pymysql://root:@localhost/csvdata01_02102024"
engine = create_engine(DATABASE_URL)

# Define query
query = "SELECT amenity_groups FROM ratehawk WHERE my_id = '3199216';"

with engine.connect() as conn:
    column = pd.read_sql_query(query, conn)
    
    # Retrieve the data if available
    if not column.empty:
        data = column.iloc[0, 0]  # First row, first column
        data_dict = {"amenity_groups": data}  # Prepare data as a dictionary for JSON
        
        # Convert the dictionary to a JSON-formatted string and print
        json_output = json.dumps(data_dict, indent=4)
        print(json_output)
    else:
        print("No matching data found.")


{
    "amenity_groups": "[{'amenities': ['Shopping on site', 'Air conditioning', 'Currency exchange', 'Smoke-free property', 'Security guard', 'Ticket assistance', 'Gift shop', 'Express check-in/check-out', 'Garden', 'Television in lobby', 'Terrace', 'Reception desk'], 'non_free_amenities': [], 'group_name': 'General'}, {'amenities': ['Bridal suite', 'Non-smoking rooms', 'Room service', 'Family room', 'Minibar', 'Toiletries'], 'non_free_amenities': [], 'group_name': 'Rooms'}, {'amenities': ['Accessibility features'], 'non_free_amenities': [], 'group_name': 'Accessibility'}, {'amenities': ['Ironing', 'Luggage storage', 'Laundry', 'Safe-deposit box', 'Concierge services', 'Dry-cleaning', 'Luggage storage', 'Telephone'], 'non_free_amenities': ['Ironing', 'Laundry', 'Dry-cleaning'], 'group_name': 'Services and amenities'}, {'amenities': ['Bar', 'Coffee/tea for guests', 'Breakfast', 'Buffet breakfast', 'Breakfast in the room', 'Cafe', 'Restaurant', 'Kitchen', 'Bottled water', 'Kettle'], 'no

In [49]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection setup
DATABASE_URL = "mysql+pymysql://root:@localhost/csvdata01_02102024"
engine = create_engine(DATABASE_URL)

# Define query
query = "SELECT amenity_groups FROM ratehawk WHERE my_id = '3199216';"

with engine.connect() as conn:
    column = pd.read_sql_query(query, conn)
    
    # Retrieve the data if available
    if not column.empty:
        data = column.iloc[0, 0]  # First row, first column
        data_dict = {"amenity_groups": data}  # Store data in a dictionary
        
        # Print the dictionary
        print(data_dict)
    else:
        print("No matching data found.")


{'amenity_groups': "[{'amenities': ['Shopping on site', 'Air conditioning', 'Currency exchange', 'Smoke-free property', 'Security guard', 'Ticket assistance', 'Gift shop', 'Express check-in/check-out', 'Garden', 'Television in lobby', 'Terrace', 'Reception desk'], 'non_free_amenities': [], 'group_name': 'General'}, {'amenities': ['Bridal suite', 'Non-smoking rooms', 'Room service', 'Family room', 'Minibar', 'Toiletries'], 'non_free_amenities': [], 'group_name': 'Rooms'}, {'amenities': ['Accessibility features'], 'non_free_amenities': [], 'group_name': 'Accessibility'}, {'amenities': ['Ironing', 'Luggage storage', 'Laundry', 'Safe-deposit box', 'Concierge services', 'Dry-cleaning', 'Luggage storage', 'Telephone'], 'non_free_amenities': ['Ironing', 'Laundry', 'Dry-cleaning'], 'group_name': 'Services and amenities'}, {'amenities': ['Bar', 'Coffee/tea for guests', 'Breakfast', 'Buffet breakfast', 'Breakfast in the room', 'Cafe', 'Restaurant', 'Kitchen', 'Bottled water', 'Kettle'], 'non_fre

In [47]:
json_data_in_column_index = json.dumps(data, indent=4)
print(json_data_in_column_index)

"[{'amenities': ['Shopping on site', 'Air conditioning', 'Currency exchange', 'Smoke-free property', 'Security guard', 'Ticket assistance', 'Gift shop', 'Express check-in/check-out', 'Garden', 'Television in lobby', 'Terrace', 'Reception desk'], 'non_free_amenities': [], 'group_name': 'General'}, {'amenities': ['Bridal suite', 'Non-smoking rooms', 'Room service', 'Family room', 'Minibar', 'Toiletries'], 'non_free_amenities': [], 'group_name': 'Rooms'}, {'amenities': ['Accessibility features'], 'non_free_amenities': [], 'group_name': 'Accessibility'}, {'amenities': ['Ironing', 'Luggage storage', 'Laundry', 'Safe-deposit box', 'Concierge services', 'Dry-cleaning', 'Luggage storage', 'Telephone'], 'non_free_amenities': ['Ironing', 'Laundry', 'Dry-cleaning'], 'group_name': 'Services and amenities'}, {'amenities': ['Bar', 'Coffee/tea for guests', 'Breakfast', 'Buffet breakfast', 'Breakfast in the room', 'Cafe', 'Restaurant', 'Kitchen', 'Bottled water', 'Kettle'], 'non_free_amenities': [], '

In [36]:
query = "SELECT amenity_groups FROM ratehawk WHERE my_id = 3199216;"

with engine.connect() as conn:
    column = pd.read_sql_query(query, conn)
    print(column)


                                      amenity_groups
0  [{'amenities': ['Shopping on site', 'Air condi...


In [ ]:
df = pd.read_sql(engine, table)